In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

bank_file = "../input/Churn_Modelling.csv"
bank_data = pd.read_csv(bank_file)

import os
print(os.listdir("../input"))

# Sıralama işlemi
bank_data = bank_data.sort_values(by=['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'])

# Any results you write to the current directory are saved as output.

# Veri Keşfi 

In [ ]:
bank_data.head() #ilk 5 satır

In [ ]:
bank_data.tail() # son 5 satır

In [ ]:
bank_data.shape # satır ve sütun sayısı

In [ ]:
bank_data.info() # bellek kullanımı ve türleri

In [ ]:
bank_data.isnull().any() # null değer varmı

In [ ]:
bank_data.describe() # basit bir istatistik

# Veri Görselleştirme

In [ ]:
# Histogram Gösterimi
num_bins=10
bank_data.hist(bins=num_bins, figsize=(20,15))

In [ ]:
plt.matshow(bank_data.corr()) # Korelasyon grafik

* İki veya daha fazla değişken arasındaki ilişkinin varlığı,
bu ilişkinin yönü ve şiddeti korelasyon analizi ile
belirlenir.
* Yukarıdaki şekilde koralasyon doğru bir şekilde dağılmış ki bize doğru sonucu gösterebilir.

In [ ]:
bank_data.corr() # Tablo halinde koralasyon

In [ ]:
# seaborn ısı haritası
import seaborn as sns
corr = bank_data.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

* Yukarıdaki ısı haritasında görüldüğü gibi doğrusal bir grafik oluşmaktadır.

In [ ]:
# Veri ön işlemeyi burada erken yaptık, ilerki adımlarda kolaylık olsun
bank_data.drop('RowNumber', axis=1, inplace = True)
bank_data.drop('CustomerId', axis=1, inplace = True)

In [ ]:
bank_data.corr() # tekrar korelasyon istatistikleri kontrol ediyoruz

In [ ]:
# Plot çizini oluşturma - 2 yüksek koralasyon değeri olan öznitelik seçtik
bank_data.plot(x='CreditScore', y='Balance', style='o')  
plt.title('CreditScore - Balance')  
plt.xlabel('CreditScore')  
plt.ylabel('Balance')  
plt.show() 

 # Veri Ön işleme
** Veri üzerinde keşfedildikten sonra;**
* Verileri türleri  int64, float64 ve object olduğu,
* Veriler üzerinde null değerlerin olmadığı,
* "csv" formatındaki dataset üzerinde 1000 satır ve 14 sütun olduğu saptanmıştır.

In [ ]:
# Dataset üzerinde bulunan verileri model çıkarmak için dönüştürme işlemi
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder() 

bank_data['Surname'] = lb.fit_transform(bank_data['Surname'])
bank_data['Geography'] = lb.fit_transform(bank_data['Geography'])
bank_data['Gender'] = lb.fit_transform(bank_data['Gender'])



In [ ]:
bank_data.sample(10) # 10 adet örnek gösterimi

In [ ]:
bank_data['Age'].max()

In [ ]:
bank_data['CreditScore'].max ()

In [ ]:
#Eksik Değer kontrolü
bank_data.isnull().any() # null Değer varmı


* Yukarıda görüldüğü gibi eksik değer olmadığından adımlar devam edildi.

In [ ]:
bank_data.isnull().sum() # Null olan öznitelikler

In [ ]:
# Yaşın aykırı uç verilerin tespiti
import seaborn as sns
sns.boxplot(x=bank_data['Age'])


In [ ]:
# Kredi puanının aykırı uç verilerin tespiti
sns.boxplot(x=bank_data['CreditScore'])

In [ ]:
# Mevcut özniteliklerden yeni bir öznitelik oluşturma
# Bu arada burada Kredi Puanına göre ev alma durumunu oluşturmuş olduk

def KrediPuani(puan):
    return (puan>550)

bank_data['EvAlma'] = bank_data['CreditScore'].apply(KrediPuani)

In [ ]:
bank_data['EvAlma'] = lb.fit_transform(bank_data['EvAlma']) # EvAlma sütununu convert ettik 
bank_data.sample(10) # sonuçlar

In [ ]:
# Balance ve CreditScore sütununu aldık 
from sklearn import preprocessing
# Normalize total_bedrooms column
x_array = np.array(bank_data['Balance'])
y_array = np.array(bank_data['CreditScore'])
normalized_X = preprocessing.normalize([x_array])
normalized_Y = preprocessing.normalize([y_array])

In [ ]:
bank_data.sample(15) # sonuçları inceleme

# Model Eğitimi

In [ ]:
# Verileri eğitim ve test verisi olarak bölüyoruz.
X = bank_data.iloc[:,0:12] 
Y = bank_data.iloc[:,11]

In [ ]:
#Eğitim ve doğrulama veri kümelerinin ayrıştırılması
from sklearn.model_selection import train_test_split  
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [ ]:
#Naive Bayes modelinin oluşturulması 1.model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

model = GaussianNB()

In [ ]:
#NB modelinin K-katlamalı çapraz doğrulama ile ACC değerinin hesaplanması
from sklearn.model_selection import KFold
scoring = 'accuracy'
kfold = KFold(n_splits=10, random_state=seed)
cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)

In [ ]:
cv_results # sonuç

In [ ]:
msg = "%f (%f)" % (cv_results.mean(), cv_results.std())
msg

# Model sonuçlarının karşılaştırılması ve yorumlanması

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

# Summary of the predictions made by the classifier
print(classification_report(Y_test, Y_pred))
print("Karmaşıklık Matrisi \n",confusion_matrix(Y_test, Y_pred))
# Accuracy score

print("Banka müşterisinin Exited(Bankadan Ayrılmama/Memnuniyet) olma olasılığı (ACC): %%%.2f" % (accuracy_score(Y_pred,Y_test)*100))

* İlk olarak ACC oranın iyi olduğunu görmekteyiz bizim için yeterli sonuç genel olarak verilerimiz tutarlı
* Precision üzerinde False=0 olan 0.81 değerin ve True=1 olan 0.46 hassasiyet değerleri görülmektedir.
* Recall üzerinde False=0 olan 0.96 değerin ve True=1 olan 0.12 geri çağırma değerleri görülmektedir.
* Karmaşıklık matrisinde doğruluk değerlerinin gerçek verilere ve öngörülen veriler oranı bize verir.